## Havvind - Integrated potential density anomaly at Sørvest-F and Nordvest-C

Potential density have been calculated for all daily reference and experiment files. Files are found in: `/lustre/storeB/project/nwp/havvind/hav/analysis_kjsta/output_mld`, with output variable named `pd`. This is the total potential density, not anomaly. It was calculated by interpolating potential temperature and salinity from s-levels to z-levels.

Potential density anomaly is already an output variable in the reference and experiment files, however they are on s-levels. As I want to integrated over depth and save some time, I will use the already interpolated potential density.

I think the integrated potential density anomaly is the same as the potential energy anomaly? In https://doi.org/10.1016/j.ocemod.2007.12.003 it is defined as 
$$\varphi = \frac{1}{H} \int_{-h}^{\eta}(\bar{\rho} - \rho) gz dz$$

"...where ρ is the vertical density profile over the water column of depth H, given by H = η + h, η the free surface, h the location of the bed, $\bar\rho$
 the depth averaged density, z the vertical co-ordinate and g the gravitational acceleration. For a given density profile, φ (J/m3) represents the amount of work required to bring about complete vertical mixing per unit of volume."

 From https://doi.org/10.3389/fmars.2025.1531684 we have the same definition, and they say "The potential energy anomaly (ϕ) and its derivative (ϕ_t) are used to evaluate the competition between mixing and stratification in the water column and the processes that cause it. ϕ explains the amount of mechanical energy (per m3) required to reach a specific density profile in the water column with a given density profile. "

 Okay my thoughts:

 My first immediate thought was to do $$\int_{-H}^{0} \rho' dz$$, but that doesn't really make much sense I think. Because what I think we want to know is if the turbines and changed wind stress alters mixing. If we look at the potential energy however, we get a feel for the mixing. Higher potential energy would be more mixed and vice versa. Meaning we would go with the equation found in the other papers above. Because if there is more or less mixing in the presence of wind farms, a change in potential energy of the column would be seen. 

Starting with making a grid, opening turbine coordinates, extracting the files containing potential density, calculating the Rossby radius and making a study area of $50R_1$ x $50R_1$ around the wind farms (where the Rossby deformation radius is the area mean of June), and getting out the z-levels which the data was interpolated to.

In [1]:
%load_ext autoreload
%autoreload 2

from Rossby_deformation.get_turbine_coords import get_turbine_coords
from Rossby_deformation.funcs import *
import matplotlib.pyplot as plt
import matplotlib.style as mplstyle
import cartopy.feature as cfeature

mplstyle.use(['ggplot', 'fast'])

In [2]:
# Making a grid
path = '/lustre/storeB/project/nwp/havvind/hav/results/experiment/EXP-03/norkyst_avg_0001.nc'
fid = Dataset(path)
grid = SGrid(fid)
del fid

In [3]:
# Opening the turbine coordinates as a xArray DataSet
sorvest_F = get_turbine_coords('/lustre/storeB/project/nwp/havvind/turbine_coordinates/windfarms_Sorvest_F.txt')
nordvest_C = get_turbine_coords('/lustre/storeB/project/nwp/havvind/turbine_coordinates/windfarms_Nordvest_C.txt')

In [4]:
# Square around wind park
min_lon_SV = np.min(sorvest_F.coordinates[:,0].values)
min_lat_SV = np.min(sorvest_F.coordinates[:,1].values)
max_lon_SV = np.max(sorvest_F.coordinates[:,0].values)
max_lat_SV = np.max(sorvest_F.coordinates[:,1].values)

area_lon_SV = [min_lon_SV, max_lon_SV]
area_lat_SV = [min_lat_SV, max_lat_SV]

In [5]:
# Square around wind park
min_lon_NV = np.min(nordvest_C.coordinates[:,0].values)
min_lat_NV = np.min(nordvest_C.coordinates[:,1].values)
max_lon_NV = np.max(nordvest_C.coordinates[:,0].values)
max_lat_NV = np.max(nordvest_C.coordinates[:,1].values)

area_lon_NV = [min_lon_NV, max_lon_NV]
area_lat_NV = [min_lat_NV, max_lat_NV]

In [6]:
# Extracting paths to files containing the computed Rossby deformation radius
# Note: I'm using the reference datasets and not the experiments
# Rossby deformation radius computed from the experiment datasets are found in output_bdr/EXP

filefolder = glob('/lustre/storeB/project/nwp/havvind/hav/analysis_kjsta/output_bdr/REF')

# Only using June because we want the largest Rossby radius
months = {
"06": 30   # June
}

files=[]  # empty list to store paths in

# building paths to contain each daily file and named thereafter
for month, days in months.items():
    for day in range(1, days + 1): 
        day_str = f"{day:04}"
        file_path = f'/REF_{month}_norkyst_avg_{day_str}_brr.nc'
        files.append(filefolder[0]+file_path)

In [7]:
# Internal Rossby radius of June - from area of wind parks
R1_june_SV = monthly_mean_area(files, grid, area_lon_SV, area_lat_SV)
R1_june_SV = R1_june_SV.gamma_r

# Internal Rossby radius of June - from area of wind parks
R1_june_NV = monthly_mean_area(files, grid, area_lon_NV, area_lat_NV)
R1_june_NV = R1_june_NV.gamma_r

In [8]:
zlevs = np.arange(0,51,1)
zlevs = np.insert(zlevs,len(zlevs),values=np.arange(52,102,2), axis =0)
zlevs = np.insert(zlevs,len(zlevs),values=np.arange(105,305,5), axis =0)
zlevs = np.insert(zlevs,len(zlevs),values=np.arange(520,1020,20), axis =0)
zlevs = np.insert(zlevs,len(zlevs),values=np.arange(1050,3050,50), axis =0)

zlevs = zlevs[np.where(zlevs<=np.max(grid.h))]
zlevs = np.array(zlevs)*-1.

In [9]:
# Extracting filepaths to MLD for the reference runs

filefolder = glob('/lustre/storeB/project/nwp/havvind/hav/analysis_kjsta/output_mld/REF')

months = {
"02": 27,  # February
"03": 31,  # March
"04": 30,  # April
"05": 31,  # May
"06": 30   # June
}

files_ref=[]  # empty list to store paths in

# building paths to contain each daily file and named thereafter
for month, days in months.items():
    for day in range(1, days + 1): 
        day_str = f"{day:04}"
        file_path = f'/REF_{month}_norkyst_avg_{day_str}_mld.nc'
        files_ref.append(filefolder[0]+file_path)

In [10]:
# Extracting filepaths to MLD for the experiments

filefolder = glob('/lustre/storeB/project/nwp/havvind/hav/analysis_kjsta/output_mld/EXP')

months = {
"02": 27,  # February
"03": 31,  # March
"04": 30,  # April
"05": 31,  # May
"06": 30   # June
}

files_exp=[]  # empty list to store paths in

# building paths to contain each daily file and named thereafter
for month, days in months.items():
    for day in range(1, days + 1): 
        day_str = f"{day:04}"
        file_path = f'/EXP_{month}_norkyst_avg_{day_str}_mld.nc'
        files_exp.append(filefolder[0]+file_path)

In [11]:
# MLD and potential dens at Sørvest-F - study area is 60R1x60R1 around wind farm.
# Area lon/lat is the extent of the study area
lon_SV, lat_SV, r_SV = make_study_area(files_ref, grid, area_lon_SV, area_lat_SV, R1_june_SV)
lon_SV, lat_SV, e_SV = make_study_area(files_exp, grid, area_lon_SV, area_lat_SV, R1_june_SV)

# MLD and potential dens at Sørvest-F - study area is 60R1x60R1 around wind farm.
# Area lon/lat is the extent of the study area
lon_NV, lat_NV, r_NV = make_study_area(files_ref, grid, area_lon_NV, area_lat_NV, R1_june_NV)
lon_NV, lat_NV, e_NV = make_study_area(files_exp, grid, area_lon_NV, area_lat_NV, R1_june_NV)

In [12]:
# Adding the zlevs as a dimension
ref_SV = r_SV.expand_dims(dim={'depth': zlevs})
exp_SV = e_SV.expand_dims(dim={'depth': zlevs})

ref_NV = r_NV.expand_dims(dim={'depth': zlevs})
exp_NV = e_NV.expand_dims(dim={'depth': zlevs})

In [13]:
# Saving some memory
del r_SV
del e_SV
del r_NV
del e_NV

Now that we have that, we have to convert the potential density to potential density anomaly.

In [14]:
pd_anom_ref_SV = ref_SV.pd - 1000
pd_anom_exp_SV = exp_SV.pd - 1000

pd_anom_ref_NV = ref_NV.pd - 1000
pd_anom_exp_NV = exp_NV.pd - 1000

Testing out how to do this on one daily file for starters.

In [15]:
# Extracting filepaths to the reference runs

filefolder = glob('/lustre/storeB/project/nwp/havvind/hav/results/reference/')

months = {
"02": 27,  # February
"03": 31,  # March
"04": 30,  # April
"05": 31,  # May
"06": 30   # June
}

files_ref=[]  # empty list to store paths in

# building paths to contain each daily file and named thereafter
for month, days in months.items():
    for day in range(1, days + 1): 
        day_str = f"{day:04}"
        file_path = f'/REF-{month}/norkyst_avg_{day_str}.nc'
        files_ref.append(filefolder[0]+file_path)

In [16]:
# Extracting filepaths to the experiment runs

filefolder = glob('/lustre/storeB/project/nwp/havvind/hav/results/experiments/')

months = {
"02": 27,  # February
"03": 31,  # March
"04": 30,  # April
"05": 31,  # May
"06": 30   # June
}

files_exp=[]  # empty list to store paths in

# building paths to contain each daily file and named thereafter
for month, days in months.items():
    for day in range(1, days + 1): 
        day_str = f"{day:04}"
        file_path = f'/EXP-{month}/norkyst_avg_{day_str}.nc'
        files_exp.append(filefolder[0]+file_path)

IndexError: list index out of range

In [ ]:
ds_ref = xr.open_dataset('/lustre/storeB/project/nwp/havvind/hav/results/experiment/EXP-03/norkyst_avg_0001.nc')
zeta = ds_ref.zeta
del ds_ref

In [ ]:
ds_ref = xr.open_dataset('/lustre/storeB/project/nwp/havvind/hav/analysis_kjsta/output_mld/EXP/EXP_03_norkyst_avg_0001_mld.nc')
ds_ref = ds_ref.expand_dims(dim={'depth': zlevs})
pd = ds_ref.pd
del ds_ref

In [ ]:
depth = grid.h
mask = grid.mask_rho
z_r = grid.z_r
g = 9.81

In [ ]:
# Integrating N over z-levels
                for k in range(0, len(N) - 1):
                    tmpn[0, y, x] = tmpn[0, y, x] + np.abs(
                        N_depth[k] - N_depth[k + 1]
                    ) / 2.0 * (N[k] + N[k + 1])


In [ ]:
pot_energy = np.ones((1, pd.shape[3], pd.shape[4]))*np.nan
pot_energy[0] = 1

for y in range(pd.shape[3]):
    for x in range(pd.shape[4]):
        if not mask[y, x]:
            continue

        tmpz = zlevs[np.where(zlevs[:] > z_r[:, y, x].min())].squeeze()
        z_mid = (tmpz[0:-1] + tmpz[1:]) / 2

        n = zeta[0, y, x]
        h = depth[y, x]
        H = n + h
        rho_mean = pd[:, 0, 0, y, x].mean(dim='depth')
        rho = pd[:, 0, 0, y, x]
        rho_diff = rho_mean - rho

        


KeyboardInterrupt: 